Import the modules


In [80]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

Import the dataset and visualise.


In [ ]:
training_set = pd.read_csv('/content/Energy Reading.csv')

#training_set = training_set.iloc[:,1:2].values

#plt.plot(training_set, label = 'Shampoo Sales Data')
plt.plot(training_set['Time'],training_set['Watt'] ,label = 'Energy')
plt.show()

**Seperate training and test data set.**

In [83]:
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

sc = MinMaxScaler()
training_data = sc.fit_transform(training_set)

seq_length = 4
x, y = sliding_windows(training_data, seq_length)

train_size = int(len(y) * 0.67)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

**LSTM CLASS CODE** 

In [84]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

**MODEL TRAINING** 

In [ ]:
num_epochs = 10000
learning_rate = 0.03

input_size = 2
hidden_size = 15
num_layers = 1

num_classes = 1

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)
#t=trainX*1.01
# Train the model
for epoch in range(num_epochs):
    outputs = lstm(trainX*1.02)
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, trainY)
    
    loss.backward()
    outputs=nn.Dropout(.5)
    optimizer.step()
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

Evaluating the model


In [ ]:
lstm.eval()
train_predict = lstm(dataX)

data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

# data_predict = sc.inverse_transform(data_predict)
# dataY_plot = sc.inverse_transform(dataY_plot)

plt.axvline(x=train_size, c='r', linestyle='--')

plt.plot(dataY_plot, label='actual')
plt.plot(data_predict,label='Predicted')
plt.suptitle('Energy Time-Series Prediction')
plt.legend(loc="upper right")
plt.show()

Saving the model and it's weights


In [87]:
torch.save(lstm.state_dict(),'Energy_prediction.pt')

In [88]:
#print(lstm.state_dict())

Loading the weights


In [89]:
# input_size = 1
# hidden_size = 2
# num_layers = 1

# num_classes = 1

# l2=LSTM(num_classes, input_size, hidden_size, num_layers)
# l2.load_state_dict(torch.load('/content/Energy_prediction.pt'))
# l2

Actual Energy consumed!

In [ ]:
y=np.array(dataY_plot)
y=y.flatten()
TW=np.sum(y)
print("The total Watt usage = ",round(TW,3),"W")

Predicted Energy Consumed!


In [ ]:
data_predict=data_predict.flatten()
TW_pred =np.sum(data_predict)
print("The total predicted Watt usage = ",round(TW_pred,3),"W")

Electricity Bill estimation

In [ ]:
x=float(input("Enter the per unit cost of electricity:"))
print("Your estimated bill is Rs",round(TW_pred*x,2))